In [1]:
import sys
import os

# 프로젝트 루트 디렉토리 경로를 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
import jax
import jax.numpy as jnp
import time
#disable jax JIT
#jax.config.update("jax_disable_jit", True)

from tqdm.autonotebook import trange
from functools import partial
from JAxtar.hash import hash_func_builder
from puzzle.slidepuzzle import SlidePuzzle
from heuristic.slidepuzzle_heuristic import SlidePuzzleHeuristic
from JAxtar.hash import HashTable

/tmp/ipykernel_29874/2078866209.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [3]:
count = int(1e6)
puzzle = SlidePuzzle(4)
hash_func = hash_func_builder(puzzle.State)
sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(0),count))
new_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(1),count))
table = HashTable.make_lookup_table(puzzle.State, 1, int(1e7))

lookup = jax.jit(partial(HashTable.lookup, hash_func))
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(time.time()-start)
#print(seeds, idx, table_idx, found)

0.1957716941833496


In [4]:
batch = 10000
parallel_insert = jax.jit(partial(HashTable.parallel_insert, hash_func))
start = time.time()
for i in trange(count // batch):
    table, _ = parallel_insert(table, sample[i*batch:(i+1)*batch])
print("insert time:", time.time()-start)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(jnp.sum(found) / count)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, new_sample)
print(jnp.sum(found) / count)

start = time.time()
for i in trange(count // batch):
    table, _ = parallel_insert(table, new_sample[i*batch:(i+1)*batch])
print("insert time:", time.time()-start)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print(jnp.sum(found) / count)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, new_sample)
print(jnp.sum(found) / count)

100%|██████████| 100/100 [00:00<00:00, 101.48it/s]


insert time: 0.9890913963317871
1.0
0.0


100%|██████████| 100/100 [00:00<00:00, 220.90it/s]


insert time: 0.4542374610900879
1.0
1.0


In [5]:
another_sample = jax.vmap(puzzle.get_initial_state)(key=jax.random.split(jax.random.PRNGKey(123),count))
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print(jnp.sum(found) / count)

0.0


In [6]:
start = time.time()
for i in trange(count // batch):
    table, _ = parallel_insert(table, another_sample[i*batch:(i+1)*batch])
print("insert time:", time.time()-start)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
start = time.time()
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, new_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)
idx, table_idx, found = jax.vmap(lookup, in_axes=(None, 0))(table, another_sample)
print("check time:", time.time()-start)
print(jnp.sum(found) / count)

100%|██████████| 100/100 [00:00<00:00, 136.18it/s]


insert time: 0.736487865447998
check time: 0.007135152816772461
1.0
check time: 0.14470362663269043
1.0
check time: 0.3515336513519287
1.0
